In [3]:
!tar -xzvf review_polarity.tar.gz

txt_sentoken/neg/cv000_29416.txt
txt_sentoken/neg/cv001_19502.txt
txt_sentoken/neg/cv002_17424.txt
txt_sentoken/neg/cv003_12683.txt
txt_sentoken/neg/cv004_12641.txt
txt_sentoken/neg/cv005_29357.txt
txt_sentoken/neg/cv006_17022.txt
txt_sentoken/neg/cv007_4992.txt
txt_sentoken/neg/cv008_29326.txt
txt_sentoken/neg/cv009_29417.txt
txt_sentoken/neg/cv010_29063.txt
txt_sentoken/neg/cv011_13044.txt
txt_sentoken/neg/cv012_29411.txt
txt_sentoken/neg/cv013_10494.txt
txt_sentoken/neg/cv014_15600.txt
txt_sentoken/neg/cv015_29356.txt
txt_sentoken/neg/cv016_4348.txt
txt_sentoken/neg/cv017_23487.txt
txt_sentoken/neg/cv018_21672.txt
txt_sentoken/neg/cv019_16117.txt
txt_sentoken/neg/cv020_9234.txt
txt_sentoken/neg/cv021_17313.txt
txt_sentoken/neg/cv022_14227.txt
txt_sentoken/neg/cv023_13847.txt
txt_sentoken/neg/cv024_7033.txt
txt_sentoken/neg/cv025_29825.txt
txt_sentoken/neg/cv026_29229.txt
txt_sentoken/neg/cv027_26270.txt
txt_sentoken/neg/cv028_26964.txt
txt_sentoken/neg/cv029_19943.txt
txt_sentoken/n

In [6]:
# @title Importing all required packages
import os
import nltk
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from collections import Counter
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('opinion_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to /root/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


True

In [7]:
# @title making data in a table of frequency of postive and negative unigrams
def calculate_freq(text, positive_criteria, negative_criteria):
    tokens = word_tokenize(text)
    pos_freq = sum(1 for token in tokens if token.lower() in positive_criteria)
    neg_freq = sum(1 for token in tokens if token.lower() in negative_criteria)
    return pos_freq, neg_freq

# Paths to your positive and negative folders
positive_folder = '/content/txt_sentoken/pos'
negative_folder = '/content/txt_sentoken/neg'

# Use NLTK's opinion_lexicon positive and negative words as criteria
positive_criteria = set(opinion_lexicon.positive())
negative_criteria = set(opinion_lexicon.negative())

# Create lists to store positive and negative frequencies for each document
positive_freq_per_doc = []
negative_freq_per_doc = []
class_labels = []

# Calculate positive and negative frequencies for each document in the positive folder
for file_name in os.listdir(positive_folder):
    with open(os.path.join(positive_folder, file_name), 'r', encoding='utf-8') as file:
        content = file.read()
        pos_freq, neg_freq = calculate_freq(content, positive_criteria, negative_criteria)
        positive_freq_per_doc.append(pos_freq)
        negative_freq_per_doc.append(neg_freq)
        class_labels.append(1)  # Class label 1 for positive documents

# Calculate positive and negative frequencies for each document in the negative folder
for file_name in os.listdir(negative_folder):
    with open(os.path.join(negative_folder, file_name), 'r', encoding='utf-8') as file:
        content = file.read()
        pos_freq, neg_freq = calculate_freq(content, positive_criteria, negative_criteria)
        positive_freq_per_doc.append(pos_freq)
        negative_freq_per_doc.append(neg_freq)
        class_labels.append(0)  # Class label 0 for negative documents

#normalizing them
positive_freq_per_doc_np = np.array(positive_freq_per_doc)
negative_freq_per_doc_np = np.array(negative_freq_per_doc)
positive_freq_per_doc_nor = positive_freq_per_doc_np / (positive_freq_per_doc_np + negative_freq_per_doc_np)
negative_freq_per_doc_nor = negative_freq_per_doc_np / (positive_freq_per_doc_np + negative_freq_per_doc_np)

# Combine positive and negative frequencies and class labels into a DataFrame
data = {
    'Positive_Freq': positive_freq_per_doc_nor,
    'Negative_Freq': negative_freq_per_doc_nor,
    'Class': class_labels
}
df = pd.DataFrame(data)

# Display the DataFrame
print(df)


      Positive_Freq  Negative_Freq  Class
0          0.406250       0.593750      1
1          0.641509       0.358491      1
2          0.595238       0.404762      1
3          0.678571       0.321429      1
4          0.750000       0.250000      1
...             ...            ...    ...
1995       0.339286       0.660714      0
1996       0.492063       0.507937      0
1997       0.425000       0.575000      0
1998       0.450000       0.550000      0
1999       0.294118       0.705882      0

[2000 rows x 3 columns]


In [12]:
# @title naive bayes with freq
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

X = df[['Positive_Freq', 'Negative_Freq']]
y = df['Class']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify = y)

# Creating a Multinomial Naive Bayes classifier
nb = MultinomialNB()

# Training the Naive Bayes model
nb.fit(X_train, y_train)

# Predicting on the test set
y_pred = nb.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Classification report
print(classification_report(y_test, y_pred))


Accuracy: 0.7083333333333334
              precision    recall  f1-score   support

           0       0.70      0.72      0.71       300
           1       0.71      0.70      0.71       300

    accuracy                           0.71       600
   macro avg       0.71      0.71      0.71       600
weighted avg       0.71      0.71      0.71       600



In [13]:
# @title SVM using freq
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

X = df[['Positive_Freq', 'Negative_Freq']]
y = df['Class']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Creating an SVM classifier
svm = SVC(kernel='linear', C=1.0, random_state=42)

# Training the SVM model
svm.fit(X_train, y_train)

# Predicting on the test set
y_pred = svm.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Classification report
print(classification_report(y_test, y_pred))


Accuracy: 0.6716666666666666
              precision    recall  f1-score   support

           0       0.66      0.69      0.68       298
           1       0.68      0.65      0.67       302

    accuracy                           0.67       600
   macro avg       0.67      0.67      0.67       600
weighted avg       0.67      0.67      0.67       600



In [11]:
# @title making a WORD format
from nltk.tokenize import word_tokenize
import pandas as pd
import os

# Define paths to your positive and negative folders
positive_folder = '/content/txt_sentoken/pos'
negative_folder = '/content/txt_sentoken/neg'

# Function to extract tokens from a file
def extract_tokens(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        tokens = word_tokenize(content)
        return tokens

# Extracting all unique tokens from multiple files
all_tokens = set()
for folder in [positive_folder, negative_folder]:
    for file in os.listdir(folder):
        file_path = os.path.join(folder, file)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            tokens = word_tokenize(content)
            all_tokens.update(tokens)

# Separate positive and negative tokens
positive_tokens = set(opinion_lexicon.positive()).intersection(all_tokens)
negative_tokens = set(opinion_lexicon.negative()).intersection(all_tokens)

# Create DataFrame columns with positive and negative tokens
columns = list(positive_tokens) + list(negative_tokens)

# Creating a DataFrame to store count of positive and negative tokens for each file
file_paths = [os.path.join(positive_folder, file) for file in os.listdir(positive_folder)] + \
             [os.path.join(negative_folder, file) for file in os.listdir(negative_folder)]
count_data = []
for file_path in file_paths:
    tokens_in_file = extract_tokens(file_path)
    positive_tokens_in_file = [token for token in tokens_in_file if token in positive_tokens]
    negative_tokens_in_file = [token for token in tokens_in_file if token in negative_tokens]
    count_vector = [positive_tokens_in_file.count(token) for token in columns[:len(positive_tokens)]] + \
                   [negative_tokens_in_file.count(token) for token in columns[len(positive_tokens):]]
    # Determine classifier (1 for positive files, 0 for negative files)
    classifier = 1 if file_path.startswith(positive_folder) else 0
    count_data.append(count_vector + [classifier])

# Convert count vectors into a DataFrame
columns.append('Class')
count_df = pd.DataFrame(count_data, columns=columns)

# Displaying the resulting DataFrame
print(count_df)


      joyful  seamless  sincerely  upscale  greatness  loyal  delicate  \
0          0         0          0        0          0      0         0   
1          0         0          0        0          0      0         0   
2          0         0          0        0          0      0         0   
3          0         0          0        0          0      0         0   
4          0         0          0        0          0      0         0   
...      ...       ...        ...      ...        ...    ...       ...   
1995       0         0          0        0          0      0         0   
1996       0         0          0        0          0      0         0   
1997       0         0          0        0          0      0         0   
1998       0         0          0        0          0      0         0   
1999       0         0          0        0          0      0         0   

      fearlessly  memorable  worthy  ...  unjust  laconic  bullies  negative  \
0              0          0    

In [12]:
# @title Dont worry ,actually this dataframe have 1's , check it here
ones_locations = count_df[count_df == 1].stack().index.tolist()

if ones_locations:
    print("Locations of '1's in the DataFrame:")
    for row, col in ones_locations:
        print(f"Row: {row}, Column: {col}")
else:
    print("There are no '1's in the DataFrame.")


Streaming output truncated to the last 5000 lines.
Row: 1870, Column: critics
Row: 1870, Column: sorry
Row: 1870, Column: stranger
Row: 1870, Column: damn
Row: 1870, Column: lame
Row: 1870, Column: unnecessary
Row: 1870, Column: problem
Row: 1870, Column: shady
Row: 1870, Column: hating
Row: 1870, Column: shock
Row: 1870, Column: awful
Row: 1870, Column: failed
Row: 1870, Column: arrogant
Row: 1870, Column: shame
Row: 1870, Column: gross
Row: 1870, Column: unpredictable
Row: 1870, Column: mercilessly
Row: 1870, Column: sick
Row: 1870, Column: killed
Row: 1870, Column: tension
Row: 1870, Column: die
Row: 1870, Column: slack
Row: 1870, Column: abruptly
Row: 1870, Column: awfully
Row: 1870, Column: underestimate
Row: 1870, Column: scared
Row: 1870, Column: hate
Row: 1870, Column: warning
Row: 1870, Column: nasty
Row: 1870, Column: extraneous
Row: 1870, Column: creepy
Row: 1871, Column: well
Row: 1871, Column: accomplish
Row: 1871, Column: honest
Row: 1871, Column: proves
Row: 1871, Column

In [42]:
# @title naive bayes with word
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report

# Splitting the data into train and test sets
X = count_df.drop('Class', axis=1)
y = count_df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initializing and fitting the Naive Bayes classifier
naive_bayes_word = MultinomialNB()
naive_bayes_word.fit(X_train, y_train)

# Predicting on the test set
y_pred = naive_bayes_word.predict(X_test)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Naive Bayes Accuracy:", accuracy)

# Classification report
print(classification_report(y_test, y_pred))

Naive Bayes Accuracy: 0.8416666666666667
              precision    recall  f1-score   support

           0       0.84      0.84      0.84       298
           1       0.84      0.84      0.84       302

    accuracy                           0.84       600
   macro avg       0.84      0.84      0.84       600
weighted avg       0.84      0.84      0.84       600



In [32]:
# @title MAXENT with word
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

X = count_df.drop('Class', axis=1)
y = count_df['Class']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Creating a Logistic Regression classifier (MaxEnt)
maxent_word = LogisticRegression(max_iter=1000)  # Increase max_iter if needed

# Training the MaxEnt model
maxent_word.fit(X_train, y_train)

# Predicting on the test set
y_pred = maxent_word.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Classification report
print(classification_report(y_test, y_pred))


Accuracy: 0.8166666666666667
              precision    recall  f1-score   support

           0       0.82      0.81      0.81       298
           1       0.82      0.82      0.82       302

    accuracy                           0.82       600
   macro avg       0.82      0.82      0.82       600
weighted avg       0.82      0.82      0.82       600



In [33]:
# @title SVM using word
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

X = count_df.drop('Class', axis=1)
y = count_df['Class']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Creating an SVM classifier
svm_word = SVC(kernel='linear', C=1.0, random_state=42)

# Training the SVM model
svm_word.fit(X_train, y_train)

# Predicting on the test set
y_pred = svm_word.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Classification report
print(classification_report(y_test, y_pred))


Accuracy: 0.7866666666666666
              precision    recall  f1-score   support

           0       0.79      0.78      0.78       298
           1       0.78      0.80      0.79       302

    accuracy                           0.79       600
   macro avg       0.79      0.79      0.79       600
weighted avg       0.79      0.79      0.79       600



In [26]:
# @title making this in TF_IDF format data
from sklearn.feature_extraction.text import TfidfVectorizer

#function for making corpus of only postive and negative tokens
def make_corpus(text, positive_criteria, negative_criteria):
    corpus = ''
    tokens = word_tokenize(text)
    for token in tokens:
       if token.lower() in (positive_criteria | negative_criteria):
          corpus += token + ' '  # Concatenate each token with a space
    return corpus.rstrip()  # Remove trailing space before returning

# Paths to your positive and negative folders
positive_folder = '/content/txt_sentoken/pos'
negative_folder = '/content/txt_sentoken/neg'

# Use NLTK's opinion_lexicon positive and negative words as criteria
positive_criteria = set(opinion_lexicon.positive())
negative_criteria = set(opinion_lexicon.negative())

# Create lists to store positive and negative frequencies for each document
corpus = []
class_labels = []

# Calculate positive and negative frequencies for each document in the positive folder
for file_name in os.listdir(positive_folder):
    with open(os.path.join(positive_folder, file_name), 'r', encoding='utf-8') as file:
        content = file.read()
        current_corpus = make_corpus(content, positive_criteria, negative_criteria)
        if current_corpus.strip():  # Check if the current corpus is not empty
            corpus.append(current_corpus)
            class_labels.append(1)  # Class label 1 for positive documents

# Calculate positive and negative frequencies for each document in the negative folder
for file_name in os.listdir(negative_folder):
    with open(os.path.join(negative_folder, file_name), 'r', encoding='utf-8') as file:
        content = file.read()
        current_corpus = make_corpus(content, positive_criteria, negative_criteria)
        if current_corpus.strip():  # Check if the current corpus is not empty
            corpus.append(current_corpus)
            class_labels.append(0)  # Class label 0 for negative documents


tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(corpus)

# Convert TF-IDF matrix to a DataFrame
feature_names = tfidf.get_feature_names_out()
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
df_tfidf['Class'] = class_labels  # Add the class labels as a column


print(df_tfidf)

      abiding  abnormal  abolish  abominable  abomination  abort  aborted  \
0         0.0       0.0      0.0         0.0          0.0    0.0      0.0   
1         0.0       0.0      0.0         0.0          0.0    0.0      0.0   
2         0.0       0.0      0.0         0.0          0.0    0.0      0.0   
3         0.0       0.0      0.0         0.0          0.0    0.0      0.0   
4         0.0       0.0      0.0         0.0          0.0    0.0      0.0   
...       ...       ...      ...         ...          ...    ...      ...   
1995      0.0       0.0      0.0         0.0          0.0    0.0      0.0   
1996      0.0       0.0      0.0         0.0          0.0    0.0      0.0   
1997      0.0       0.0      0.0         0.0          0.0    0.0      0.0   
1998      0.0       0.0      0.0         0.0          0.0    0.0      0.0   
1999      0.0       0.0      0.0         0.0          0.0    0.0      0.0   

      abound  abounds  abrasive  ...  zapped  zeal  zealot  zealous  \
0   

In [27]:
# @title Dont worry actually this data frame have values > 0, check it here
ones_locations = df_tfidf[df_tfidf > 0].stack().index.tolist()

if ones_locations:
    print("Locations of '1's in the DataFrame:")
    for row, col in ones_locations:
        print(f"Row: {row}, Column: {col}")
else:
    print("There are no '1's in the DataFrame.")


Streaming output truncated to the last 5000 lines.
Row: 1888, Column: benefit
Row: 1888, Column: better
Row: 1888, Column: brave
Row: 1888, Column: chill
Row: 1888, Column: clear
Row: 1888, Column: clearer
Row: 1888, Column: clearly
Row: 1888, Column: complex
Row: 1888, Column: confused
Row: 1888, Column: confusing
Row: 1888, Column: crashing
Row: 1888, Column: credible
Row: 1888, Column: demise
Row: 1888, Column: desert
Row: 1888, Column: destiny
Row: 1888, Column: disaster
Row: 1888, Column: disastrous
Row: 1888, Column: disown
Row: 1888, Column: dissatisfaction
Row: 1888, Column: disturbing
Row: 1888, Column: eagerly
Row: 1888, Column: embarrassment
Row: 1888, Column: enough
Row: 1888, Column: fabricate
Row: 1888, Column: failed
Row: 1888, Column: fails
Row: 1888, Column: fans
Row: 1888, Column: fault
Row: 1888, Column: flaws
Row: 1888, Column: flourish
Row: 1888, Column: freedom
Row: 1888, Column: frustrated
Row: 1888, Column: gentle
Row: 1888, Column: good
Row: 1888, Column: great

In [43]:
# @title naive bayes with TF_IDF
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report

# Splitting the data into train and test sets
X = df_tfidf.drop('Class', axis=1)
y = df_tfidf['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initializing and fitting the Naive Bayes classifier
naive_bayes_tfidf = MultinomialNB()
naive_bayes_tfidf.fit(X_train, y_train)

# Predicting on the test set
y_pred = naive_bayes_tfidf.predict(X_test)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Naive Bayes Accuracy:", accuracy)

# Classification report
print(classification_report(y_test, y_pred))

Naive Bayes Accuracy: 0.8333333333333334
              precision    recall  f1-score   support

           0       0.83      0.83      0.83       298
           1       0.83      0.84      0.83       302

    accuracy                           0.83       600
   macro avg       0.83      0.83      0.83       600
weighted avg       0.83      0.83      0.83       600



In [46]:
# @title MAXENT with TF-IDF
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

X = df_tfidf.drop('Class', axis=1)
y = df_tfidf['Class']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Creating a Logistic Regression classifier (MaxEnt)
maxent_tfidf = LogisticRegression(max_iter=1000)  # Increase max_iter if needed

# Training the MaxEnt model
maxent_tfidf.fit(X_train, y_train)

# Predicting on the test set
y_pred = maxent_tfidf.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Classification report
print(classification_report(y_test, y_pred))


Accuracy: 0.8366666666666667
              precision    recall  f1-score   support

           0       0.84      0.83      0.83       298
           1       0.83      0.85      0.84       302

    accuracy                           0.84       600
   macro avg       0.84      0.84      0.84       600
weighted avg       0.84      0.84      0.84       600



In [47]:
# @title SVM using TF-IDF
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

X = df_tfidf.drop('Class', axis=1)
y = df_tfidf['Class']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Creating an SVM classifier
svm_tfidf = SVC(kernel='linear', C=1.0, random_state=42)

# Training the SVM model
svm_tfidf.fit(X_train, y_train)

# Predicting on the test set
y_pred = svm_tfidf.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Classification report
print(classification_report(y_test, y_pred))


Accuracy: 0.8233333333333334
              precision    recall  f1-score   support

           0       0.82      0.83      0.82       298
           1       0.83      0.82      0.82       302

    accuracy                           0.82       600
   macro avg       0.82      0.82      0.82       600
weighted avg       0.82      0.82      0.82       600



<h2>See not all models are used for classification
one such example is Word2Vec itself doesn't perform classification. It's a technique used for generating word embeddings to represent words in a continuous vector space where words with similar meanings have closer representations</h2>


In [71]:
# @title taking word2vec form
from gensim.models import Word2Vec

non_empty_corpus = [text for text in corpus if text.strip()]
y = [label for text, label in zip(corpus, class_labels) if text.strip()]
# Tokenize sentences into words
tokenized_corpus = [word_tokenize(sentence.lower()) for sentence in non_empty_corpus]

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_corpus, vector_size=100, window=5, min_count=1, workers=4)

# Create word vectors for the corpus
word_vectors = model.wv

# Forming X (using average word vectors for each sentence)
X = []
for sentence in tokenized_corpus:
    sentence_vector = [word_vectors[word] for word in sentence if word in word_vectors]
    if sentence_vector:
        avg_vector = sum(sentence_vector) / len(sentence_vector)
        X.append(avg_vector)

# X is now your training data, each element representing a sentence


binomial doesnt word on negative values

In [72]:
# @title SVM using word2vec
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Creating an SVM classifier
svm_word2vec = SVC(kernel='linear', C=1.0, random_state=42)

# Training the SVM model
svm_word2vec.fit(X_train, y_train)

# Predicting on the test set
y_pred = svm_word2vec.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Classification report
print(classification_report(y_test, y_pred))


Accuracy: 0.5866666666666667
              precision    recall  f1-score   support

           0       0.57      0.71      0.63       298
           1       0.62      0.46      0.53       302

    accuracy                           0.59       600
   macro avg       0.59      0.59      0.58       600
weighted avg       0.59      0.59      0.58       600



In [73]:
# @title MAXENT with word2vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Creating a Logistic Regression classifier (MaxEnt)
maxent_word2vec = LogisticRegression(max_iter=1000)  # Increase max_iter if needed

# Training the MaxEnt model
maxent_word2vec.fit(X_train, y_train)

# Predicting on the test set
y_pred = maxent_word2vec.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Classification report
print(classification_report(y_test, y_pred))


Accuracy: 0.59
              precision    recall  f1-score   support

           0       0.58      0.64      0.61       298
           1       0.60      0.54      0.57       302

    accuracy                           0.59       600
   macro avg       0.59      0.59      0.59       600
weighted avg       0.59      0.59      0.59       600

